<a href="https://colab.research.google.com/github/OrionXV/InterIITAdobe/blob/main/task2/ImageCaptionNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv("somepath.csv")
df['img_caption'] = None

In [ ]:
df.head()

In [ ]:
assert torch.cuda.is_available()

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to('cuda')

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

In [ ]:
# conditional image captioning
def image_captioner(img_path:str) -> str:
    raw_image = Image.open(img_path).convert('RGB')

    # unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to('cuda', torch.float16)

    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a picture of a woman sitting on the beach with a dog
woman sitting on the beach with her dog and a cell phone


In [ ]:
%time
for i in tqdm(range(len(df))):
    df['img_caption'][i] = image_captioner(df['img_path'][i])

In [ ]:
df.to_csv('captioned_dataset.csv')